# Cartographie des Musées de Paris : Exploration et Visualisation


Le présent notebook s'inscrit dans le cadre d'une analyse de données portant sur la localisation et les informations administratives des musées de France. Les données utilisées proviennent du dataset "Liste et Localisation des Musées de France" disponible sur le site data.culture.gouv.fr, accessible à l'URL : https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/liste-et-localisation-des-musees-de-france/exports/parquet?lang=fr&timezone=Europe%2FBerlin. Ce dataset fournit des informations détaillées sur les musées, notamment leur localisation géographique, leurs coordonnées, et d'autres détails administratifs.


L'objectif de cette analyse est d'adapter ces données pour créer une carte interactive des musées limités à Paris, offrant ainsi une visualisation intuitive de la répartition de ces institutions culturelles dans la capitale. Cette cartographie permettra non seulement de visualiser la densité des musées à Paris mais aussi de fournir des informations détaillées sur chaque musée à travers des marqueurs interactifs.


## Méthodologie


La méthodologie employée dans ce notebook combine l'utilisation de requêtes SQL via DuckDB pour filtrer les données et ne conserver que les musées situés à Paris, avec des techniques de visualisation de données utilisant Folium pour créer une carte interactive. La première étape a consisté à charger le dataset complet, puis à appliquer une requête SQL pour sélectionner les musées dont la commune est 'Paris', en retenant les colonnes pertinentes telles que le nom officiel du musée, la latitude et la longitude. Les résultats ont ensuite été utilisés pour créer une carte centrée sur Paris, avec des marqueurs représentant les musées, assortis de popups contenant des informations détaillées sur chaque institution.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [2]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/liste-et-localisation-des-musees-de-france/exports/parquet?lang=fr&timezone=Europe%2FBerlin", safe_mode=True)
print("✅ Données chargées avec succès")

✅ Données chargées avec succès


## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [3]:
# Exécution de la requête
df = run_query(""" SELECT 
  "nom_officiel_du_musee" AS nom_musee,
  "latitude" AS lat,
  "longitude" AS lon,
  ST_AsText("geolocalisation") AS geom_wkt
FROM 
  loaded_dataset
WHERE 
  "commune" = 'Paris'
ORDER BY 
  "nom_officiel_du_musee" """)
print(f"Résultats : {len(df)} lignes")
df.head()

Résultats : 50 lignes


,nom_musee,lat,lon,geom_wkt
0,"Palais Galliera, musée de la mode et de la vil...",48.866137,2.297248,POINT (2.297248499625773 48.866136805463356)
1,"Petit Palais, musée des beaux-arts de la ville...",48.866090,2.313763,POINT (2.313763 48.86609)
2,maison de Balzac,48.855398,2.280805,POINT (2.280805 48.855398)
3,maison de Victor Hugo,48.854808,2.366232,POINT (2.366232 48.854808)
4,musée Bourdelle,48.843017,2.318830,POINT (2.31883 48.843017)


## 📈 Visualisation

La bibliothèque principale utilisée est Folium, qui est particulièrement adaptée pour la visualisation de données géospatiales sur des cartes interactives. Folium permet de créer des cartes avec des marqueurs personnalisés et des popups riches, ce qui est idéal pour représenter des informations liées à des lieux spécifiques, comme les musées à Paris dans cet exemple. Cela rend l'information plus accessible et interactive pour l'utilisateur.

In [4]:
import pandas as pd
import duckdb as ddb
import folium
from folium.plugins import Fullscreen
import branca.colormap as cm

# Palette de couleurs musée (tons ocre/beige/café)
palette = ["#8B4513", "#A0522D", "#CD853F", "#DEB887", "#F5DEB3", "#FFE4B5"]

# Créer la carte centrée sur Paris avec un thème sobre
carte = folium.Map(
    location=[48.8567, 2.3508],
    zoom_start=12,
    tiles='CartoDB.Positron',
    attr='© OpenStreetMap contributors © CARTO'
)

# Ajouter des marqueurs avec icône personnalisée
museum_icon = folium.Icon(
    color='darkgreen',
    icon='building-columns',
    prefix='fa'
)

# Ajouter les marqueurs avec popup riche
for _, row in df.iterrows():
    popup_html = f"""
    <div style="font-family: Georgia, serif; font-size: 12px; max-width: 250px;">
        <h4 style="color: #8B4513; margin: 0 0 5px 0;">{row['nom_musee']}</h4>
        <p style="margin: 2px 0;"><b>Latitude:</b> {row['lat']:.4f}</p>
        <p style="margin: 2px 0;"><b>Longitude:</b> {row['lon']:.4f}</p>
    </div>
    """
    
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=folium.Popup(popup_html, max_width=300),
        icon=museum_icon
    ).add_to(carte)

# Ajouter le bouton plein écran
Fullscreen(position='topright').add_to(carte)

# Ajouter un cadre décoratif autour
css = """
<style>
    .folium-map {
        border: 3px solid #8B4513;
        border-radius: 10px;
        box-shadow: 0 4px 8px rgba(0,0,0,0.2);
    }
</style>
"""
carte.get_root().header.add_child(folium.Element(css))

# Enregistrer la carte
carte.save('carte_musees_paris_ambiance.html')

dataviz = carte
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*

In [5]:

# --- Variables injectées par le script ---
FINAL_OBJECT_VARIABLE_NAME = 'dataviz'
OUTPUT_IMAGE_NAME = 'published\\notebooks\\duckit_analysis_20250806_065638.png'
OUTPUT_HTML_NAME = 'published\\notebooks\\duckit_analysis_20250806_065638.html'

# ===================================================================
# CELLULE INJECTÉE AUTOMATIQUEMENT (VERSION ROBUSTE)
# ===================================================================
import sys
import os
# On importe les modules nécessaires pour l'export au cas où
try:
    from bokeh.io import save as bokeh_save
except ImportError:
    bokeh_save = None

try:
    # On s'assure que le dossier de sortie existe
    output_dir = os.path.dirname(OUTPUT_IMAGE_NAME)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # On utilise globals().get() pour une récupération plus sûre
    final_object = globals().get(FINAL_OBJECT_VARIABLE_NAME)

    if final_object is None:
        # On lève une NameError pour être cohérent avec le code original
        raise NameError(f"name '{FINAL_OBJECT_VARIABLE_NAME}' is not defined")

    print(f"INFO: Variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée. Tentative d'exportation...")

    object_type = str(type(final_object))

    if 'plotly.graph_objs._figure.Figure' in object_type:
        print(f"--> Détecté : Plotly. Sauvegarde HTML et PNG.")
        # 1. Sauvegarde HTML pour l'interactivité
        print(f"--> Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.write_html(OUTPUT_HTML_NAME, include_plotlyjs='cdn')
        # 2. Sauvegarde PNG pour l'aperçu statique
        try:
            print(f"--> Tentative de sauvegarde PNG directe dans : {OUTPUT_IMAGE_NAME}")
            final_object.write_image(OUTPUT_IMAGE_NAME, scale=3, width=1200, height=800)
            print(f"--> Image Plotly sauvegardée avec succès.")
        except Exception as e:
            print(f"AVERTISSEMENT: La sauvegarde directe en PNG a échoué (kaleido est-il installé?).", file=sys.stderr)
            print(f"   Erreur: {e}", file=sys.stderr)
            print(f"--> PLAN B: On va utiliser la capture d'écran du HTML à la place.")
            # On crée un fichier marqueur pour que le script de post-traitement prenne le relais
            with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
                f.write("plotly")
    elif 'folium.folium.Map' in object_type:
        print(f"--> Détecté : Folium. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("folium")
    elif 'altair.vegalite' in object_type and hasattr(final_object, 'save'):
        print(f"--> Détecté : Altair. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("altair")
    elif 'bokeh.plotting' in object_type and bokeh_save is not None:
        print(f"--> Détecté : Bokeh. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("bokeh")
    elif 'matplotlib.figure.Figure' in object_type:
        print(f"--> Détecté : Matplotlib. Sauvegarde dans : {OUTPUT_IMAGE_NAME}")
        final_object.savefig(OUTPUT_IMAGE_NAME, dpi=300, bbox_inches='tight')
    else:
        print(f"AVERTISSEMENT: Type non supporté : {object_type}", file=sys.stderr)
except NameError:
    print(f"AVERTISSEMENT: Aucune variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée.", file=sys.stderr)
except Exception as e:
    print(f"ERREUR lors de l'exportation : {e}", file=sys.stderr)


INFO: Variable 'dataviz' trouvée. Tentative d'exportation...
--> Détecté : Folium. Sauvegarde HTML dans : published\notebooks\duckit_analysis_20250806_065638.html
--> Création du marqueur de capture d'écran.
